In [15]:

!pip3 install --upgrade gdown --quiet
#!gdown 16GYHdSWS3iMYwMPv5FpeDZN2rH7PR0F2 # this is the file ID of miniplaces dataset
!gdown 1YXp7HwhtdkU7F11QTZXsTPrT9jq3LDAn
# back-up commands (try the following it previous file id is overload)
# !gdown 1CyIQOJienhNITwGcQ9h-nv8z6GOjV2HX


Downloading...
From: https://drive.google.com/uc?id=1YXp7HwhtdkU7F11QTZXsTPrT9jq3LDAn
To: /content/dataset.tar.gz
100% 28.5M/28.5M [00:00<00:00, 51.1MB/s]


In [16]:
import os
import tarfile
from tqdm import tqdm
import urllib.request

def setup(folder_name='Week7'):
  # Let's make our assignment directory
  CS188_path = './'
  os.makedirs(os.path.join(CS188_path, 'Week7', 'data'), exist_ok=True)
  # Now, let's specify the assignment path we will be working with as the root.
  root_dir = os.path.join(CS188_path, 'Week7')
  # Open the tar.gz file
  tar = tarfile.open("dataset.tar.gz", "r:gz")
  # Extract the file "./Assignment2/data" folder
  total_size = sum(f.size for f in tar.getmembers())
  with tqdm(total=total_size, unit="B", unit_scale=True, desc="Extracting tar.gz file") as pbar:
      for member in tar.getmembers():
          tar.extract(member, os.path.join(root_dir, 'data'))
          pbar.update(member.size)
  # Close the tar.gz file
  tar.close()
  return root_dir

In [17]:

root_dir = setup(folder_name='Week7')

Extracting tar.gz file: 100%|██████████| 28.4M/28.4M [00:00<00:00, 89.7MB/s]


### Define the data transform


In [2]:
from torchvision import transforms

# Define data transformation
# You can copy your data transform from Assignment2. 
# Notice we are resize images to 128x128 instead of 64x64.
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


### Define the dataset and dataloader

In [3]:

import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class Metadrive(Dataset):
    def __init__(self, root_dir, split, transform=None):
        
        assert split in ['train', 'val', 'test']
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.filenames = []

        count = 0
        # Iterate directory
        for path in os.listdir(os.path.join(root_dir, 'dataset', split)):
          if os.path.isfile(os.path.join(root_dir, 'dataset', split, path)):
            self.filenames.append(path)

    def __len__(self):
        
        return len(self.filenames)

    def __getitem__(self, idx):
        '''
        Label returned is of the form: tuple((float)steering, (float)acceleration, (float)velocity)
        '''
        image = None
        label = None

        image_path = os.path.join(self.root_dir, 'dataset', self.split, self.filenames[idx])
        image = Image.open(image_path)
        if self.transform:
          image = self.transform(image)
        label = self.filenames[idx][1:-5]
        label = label.split(', ')
        steering = float(label[0])*100
        acceleration = float(label[1])*10
        return image, torch.Tensor([steering, acceleration])


### Define the train method

In [4]:
def train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs):
    """
    Train the MLP classifier on the training set and evaluate it on the validation set every epoch.
    
    Args:
        model (MLP): MLP classifier to train.
        train_loader (torch.utils.data.DataLoader): Data loader for the training set.
        val_loader (torch.utils.data.DataLoader): Data loader for the validation set.
        optimizer (torch.optim.Optimizer): Optimizer to use for training.
        criterion (callable): Loss function to use for training.
        device (torch.device): Device to use for training.
        num_epochs (int): Number of epochs to train the model.
    """
    # Place model on device
    model = model.to(device)

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
    
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        # Use tqdm to display a progress bar during training
        with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}') as pbar:
            for inputs, labels in train_loader:
                # Move inputs and labels to device
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero out gradients
                optimizer.zero_grad()
                
                # Compute the logits and loss
                logits = model(inputs)
                loss = criterion(logits, labels)
                
                # Backpropagate the loss
                loss.backward()
                
                # Update the weights
                optimizer.step()
                
                # Update the progress bar
                pbar.update(1)
                pbar.set_postfix(loss=loss.item())
          
        scheduler.step()
        
        # Evaluate the model on the validation set
        avg_loss = evaluate(model, val_loader, criterion, device)
        print(f'Validation set: Average loss = {avg_loss:.4f}')

def evaluate(model, test_loader, criterion, device):
    """
    Evaluate the MLP classifier on the test set.
    
    Args:
        model (MLP): MLP classifier to evaluate.
        test_loader (torch.utils.data.DataLoader): Data loader for the test set.
        criterion (callable): Loss function to use for evaluation.
        device (torch.device): Device to use for evaluation.
        
    Returns:
        float: Average loss on the test set.
        float: Accuracy on the test set.
    """
    model.eval()  # Set model to evaluation mode
    
    with torch.no_grad():
        total_loss = 0.0
        
        for inputs, labels in test_loader:
            # Move inputs and labels to device
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Compute the logits and loss
            logits = model(inputs)
            loss = criterion(logits, labels)
            total_loss += loss.item()
            
            
    # Compute the average loss and accuracy
    avg_loss = total_loss / len(test_loader)
    
    return avg_loss

In [5]:
# Also, seed everything for reproducibility
# code from https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964#file-seed_everything-py
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [6]:
# Define the device to use for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == torch.device('cuda'):
    print(f'Using device: {device}. Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')

Please set GPU via Edit -> Notebook Settings.


In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import cv2
import torchvision.models as models
class Resnet(nn.Module):
    def __init__(self, mode='linear',pretrained=True):
        super().__init__()
        """
        use the resnet18 model from torchvision models. Remember to set pretrained as true
        
        mode has three options:
        1) features: to extract features only, we do not want the last fully connected layer of 
            resnet18. Use nn.Identity() to replace this layer.
        2) linear: For this model, we want to freeze resnet18 features, then train a linear 
            classifier which takes the features before FC (again we do not want 
            resnet18 FC). And then write our own FC layer: which takes in the features and 
            output scores of size 100 (because we have 100 categories).
            Because we want to freeze resnet18 features, we have to iterate through parameters()
            of our model, and manually set some parameters to requires_grad = False
            Or use other methods to freeze the features
        3) finetune: Same as 2), except that we we do not need to freeze the features and
           can finetune on the pretrained resnet model.
        """
        self.resnet = None
        self.resnet = models.resnet18(pretrained = pretrained)

        if mode == 'feature':
          self.resnet.fc = nn.Identity()
        
        if mode == 'linear':
          for param in self.resnet.parameters():
            param.requires_grad = False
          self.resnet.fc = nn.Linear(512, 2)

        if mode == 'finetune':
          for param in self.resnet.parameters():
            param.requires_grad = True
          self.resnet.fc = nn.Linear(512, 2)
    #####################################################################################

    def forward(self, x):
        x = self.resnet(x)
        return x
    
    def to(self,device):
        return self.resnet.to(device=device)


In [13]:
from tqdm import tqdm
seed_everything(0)

# Define the model, optimizer, and criterion (loss_fn)
model = Resnet(mode='linear',pretrained=True)

optimizer = torch.optim.SGD(
    model.parameters(), 
    lr=0.001, 
    momentum=0.9)


criterion = nn.MSELoss()


# Define the dataset and data transform with flatten functions appended
# data_dir = os.path.join('.', 'dataset')
data_dir = '.'

train_dataset = Metadrive(
    root_dir=data_dir, split='train', 
    transform=data_transform)

val_dataset = Metadrive(
    root_dir=data_dir, split='val', 
    transform=data_transform)

# Define the batch size and number of workers
batch_size = 64
num_workers = 0

# Define the data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

# Train the model
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=20)

Epoch 1/20: 100%|██████████| 15/15 [00:17<00:00,  1.17s/it, loss=16.9]


Validation set: Average loss = 19.7127


Epoch 2/20: 100%|██████████| 15/15 [00:17<00:00,  1.19s/it, loss=12.3]


Validation set: Average loss = 15.8314


Epoch 3/20: 100%|██████████| 15/15 [00:18<00:00,  1.21s/it, loss=13.4]


...

Validation set: Average loss = 16.1732


Epoch 19/20: 100%|██████████| 15/15 [00:17<00:00,  1.14s/it, loss=13.6]


Validation set: Average loss = 16.0973


Epoch 20/20: 100%|██████████| 15/15 [00:16<00:00,  1.12s/it, loss=9.52]


Validation set: Average loss = 16.0146


In [17]:
PATH = "model.pt"
torch.save({
    # 'epoch': EPOCH,
    'model_state_dict': model.state_dict(),
    # 'optimizer_state_dict': optimizer.state_dict(),
    # 'loss': LOSS,
}, PATH)

In [18]:
"""
Please feel free to run this script to enjoy a journey by keyboard!
Remember to press H to see help message!
Note: This script require rendering, please following the installation instruction to setup a proper
environment that allows popping up an window.
"""

print(model)

import random
import os
import numpy as np
from PIL import Image
from metadrive import MetaDriveEnv
from metadrive.constants import HELP_MESSAGE

from metadrive.component.algorithm.blocks_prob_dist import PGBlockDistConfig
from metadrive.component.map.base_map import BaseMap
from metadrive.component.map.pg_map import MapGenerateMethod

from rgb_policy import RGBPolicy


PRINT_IMG = False
SAMPLING_INTERVAL = 10


config = dict(
    # controller="joystick",
    use_render=True,
    offscreen_render=True,
    manual_control=True,
    traffic_density=0,
    environment_num=100,
    random_agent_model=False,
    start_seed=random.randint(0, 1000),
    vehicle_config = dict(image_source="rgb_camera", 
                            rgb_camera= (128 , 128),
                            stack_size=1),
    block_dist_config=PGBlockDistConfig,
    random_lane_width=True,
    random_lane_num=False,
    map_config={
        BaseMap.GENERATE_TYPE: MapGenerateMethod.BIG_BLOCK_SEQUENCE,
        BaseMap.GENERATE_CONFIG: "SCCCSCCC",  # it can be a file path / block num / block ID sequence
        BaseMap.LANE_WIDTH: 3.5,
        BaseMap.LANE_NUM: 1,
        "exit_length": 50,
    },
    agent_policy=RGBPolicy
)
env = MetaDriveEnv(config)
try:
    o = env.reset()
    print(HELP_MESSAGE)
    env.vehicle.expert_takeover = True
    assert isinstance(o, dict)
    print("The observation is a dict with numpy arrays as values: ", {k: v.shape for k, v in o.items()})
    #for i in range(1, 31):
    for i in range(1, 100000000000):
        o, r, d, info = env.step([0, 0])
        
        # Action space is of form (float, float) -> Tuple
        # It encodes the necessary info about vehicle movement
        action_space = (info['steering'], info['acceleration'], info['velocity'])
        print(action_space)

        # Change PRINT_IMG to True if recording FPV
        # Change step_size to set sampling rate
        # Image saved is named by the action
        if PRINT_IMG and i%SAMPLING_INTERVAL == 0:
            img = np.squeeze(o['image'][:,:,:,0]*255).astype(np.uint8)
            img = img[...,::-1].copy() # convert to rgb
            img = Image.fromarray(img)
            root_dir = os.path.join(os.getcwd(), 'dataset', 'val')
            img_path = os.path.join(root_dir, str(action_space) + ".png")
            img.save(str(img_path))

        env.render(
            text={
                "Auto-Drive (Switch mode: T)": "on" if env.current_track_vehicle.expert_takeover else "off",
            }
        )
        if d and info["arrive_dest"]:
            env.reset()
            env.current_track_vehicle.expert_takeover = True
except Exception as e:
    raise e
finally:
    env.close()

Resnet(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

TypeError: __init__() missing 1 required positional argument: 'model'